In [ ]:
%pip install langchain langchain-openai langchain-chroma python-dotenv

In [ ]:
import gradio as gr # oh yeah!
import uuid

In [ ]:
import time
import json
import os
from IPython.display import clear_output
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from enum import Enum

# Inline credentials
with open('config.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "default" 
    }    

    for key, value in env_vars.items():
            os.environ[key] = value    

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

In [ ]:
from gen_ai_hub.orchestration.utils import load_text_file 

# Load the CV file content 
cv_file_path = "cv.txt" # Specify the correct path to the CV file 
cv_content = load_text_file(cv_file_path) 

# Print the content to verify it has been loaded 
print(cv_content) 

In [ ]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage 
from gen_ai_hub.orchestration.models.template import Template, TemplateValue 

system_message = """You are a helpful AI assistant for HR. Summarize the following CV in 10 sentences,focusing on key qualifications, work experience, and achievements. Include personal contact information,  
#organizational history, and personal interests"""

# Define the template for resume screening 
template = Template( 
messages=[ 
SystemMessage("""You are a helpful AI assistant."""), 

UserMessage( 
"""{{?prompt}}
Here is a candidate's resume: {{?candidate_resume}}"""
), 
],


defaults=[ 
TemplateValue(name="candidate_resume", value="John Doe's resume content goes here..."),
TemplateValue(name="prompt", value="Hi") 
], 
) 

In [ ]:
template2 = Template( 
messages=[ 
SystemMessage("""You are a helpful AI assistant."""), 

UserMessage( 
"""{{?prompt}}"""
), 
],


defaults=[ 
#TemplateValue(name="candidate_resume", value="John Doe's resume content goes here..."),
TemplateValue(name="prompt", value="What is the candidate's contact details in the resume?") 
], 
) 

In [ ]:
AI_API_URL = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dcecde8d10aeb6a2"

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM 
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
model = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6})

In [ ]:
from gen_ai_hub.orchestration.service import OrchestrationService 
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(query, prompt):
 print(query)
 messages= [
    {"role":"system", "content": system_message},
    {"role":"user", "content": query} ] 
 
    #Choose the prompt
 if prompt == 0:
    templates = template2
        #Set the template values
 else:
    templates = template
        #Set the template values
 template_values=[
                #TemplateValue(name="candidate_resume", value=cv_content),
                TemplateValue(name="prompt", value=query),
            ]    
    
    # Create orchestration config
 config = OrchestrationConfig( 
    template=templates,
    llm=model, 
    )

 orchestration_service = OrchestrationService(AI_API_URL, config=config) 


    #Run orchestration with the provided input (for example, candidate resume content) 
 result = orchestration_service.run(template_values=template_values) 



    # Extract the response content 
 return result.orchestration_result.choices[0].message.content 
   

# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
#def shout(text):
 #   print(f"Shout has been called with input {text}")
  #  return text.upper()
#gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

#Build UI
ui = gr.Interface(
        fn=message_gpt,
        inputs=[
            gr.Textbox(label="Query"),
            gr.Dropdown(["What is the candidate's contact details in the resume?", "promot2"], label="Prompts", info="Use the built in prompt templates", type="index")],
        outputs="textbox",
        flagging_mode="never",
        js=force_dark_mode
)
ui.launch()


#Simple UI
#gr.Interface(fn=message_gpt, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
from typing import List

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService, context):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}} {{?context}}"),
                ],
            ),
            llm=model#LLM(name="gpt-4"),
        )
        self.history: List[Message] = []
        self.context = context  # Context to be used in the chat

    def chat(self, user_input):
        print(self.history)
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
                TemplateValue(name="context", value=self.context)
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

In [ ]:
orchestration_service = OrchestrationService(AI_API_URL) 
bot = ChatBot(orchestration_service=orchestration_service)
#gr.Interface(fn=bot.chat, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

** A little fancy chatbot**

In [ ]:
# import random
# import time

# #For uploads
# from pathlib import Path

# def upload_file(filepath):
#     name = Path(filepath).name
#     print(f"File uploaded: {name}")
    
    
# def process_files(files):
#     contents = []
#     for file in files:
#         with open(file.name, 'r') as f:
#             contents.append(f.read())
#     return "\n".join(contents)    
#     # vs = createChunks(files)


# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(type="messages")
#     msg = gr.Textbox()
#     clear = gr.ClearButton([msg, chatbot])

#     def respond(message, chat_history):
#         bot_message = bot.chat(message)#random.choice(["How are you?", "Today is a great day", "I'm very hungry"])
#         chat_history.append({"role": "user", "content": message})
#         chat_history.append({"role": "assistant", "content": bot_message})
#         #time.sleep(2)
#         return "", chat_history

#     msg.submit(respond, [msg, chatbot], [msg, chatbot])

#     # Upload
#     # with gr.Row():
#     #     u = gr.UploadButton("Upload files", file_count="multiple")
#     # u.upload(upload_file, u)

#     fn=process_files,
#     u =gr.Files(label="Upload multiple files")
#     u.upload(fn, u)
    
# demo.launch()


Another try...

In [ ]:


import os
import glob
from dotenv import load_dotenv
import gradio as gr

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="text-embedding-ada-002") -> str:
    response = embeddings.create(
      model_name=model,
      input=input
    )
    return response.data[0].embedding

In [ ]:
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

text = 'Every decoding is another encoding.'

embeddings = init_embedding_model('text-embedding-ada-002')
response = embeddings.embed_query(text)
print(response)

In [ ]:
db_name = "vector_db"
vectorstore = Chroma(
    collection_name=db_name,
    embedding_function=embeddings,
    persist_directory=f"./{db_name}"
)

In [ ]:
def createChunks(documents):
    embeddings = init_embedding_model('text-embedding-ada-002')
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    global namespace
    namespace = str(uuid.uuid4())
    # print(f"Using namespace2: {namespace}")
    loaders = [TextLoader(x) for x in documents]
    pages = []
    for loader in loaders:
        pages.extend(loader.load())
    
    chunks = text_splitter.split_documents(pages)

    print(chunks)

    # Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
    # Chroma is a popular open source Vector Database based on SQLLite
    
    
    
    
    # Delete if already exists
    
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
        print(f"Deleted collection {db_name}")
    else:
        print("failed to delete collection, directory does not exist")
    
    # Create vectorstore
    
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
    print(f"Vectorstore created with {vectorstore._collection.count()} documents")
    return vectorstore

In [ ]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')

chat_llm = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)

In [ ]:
prompt1 = "As an SAP solution expert , the goal is to generate comprehensive meeting notes for service delivery report that will be shared with the customer from  a file containing the complete meeting recording transcript (meeting transcript). It should includes all details from all discussions, organized by topic. The resulting detailed meeting notes should include all details for all topics discussed so that someone who was not present at the meeting can understand both the context of the topic and have all supporting details organized and grouped in logically related topics without having read the full transcript. Deliver the detailed notes using a professional and neutral tone and in an easy-to-read format by using headlines, subheads, and bullet points.  Include any meeting participant introductions at the top of the meeting notes and include all details from the introduction that are provided including name, role or title, history, background, location, and any other specific information provided. Also include a list of meeting participants and their roles if these can be determined in the meeting transcript at the top of the meeting notes, grouped by the participants stated or determined organization. At the end of the meeting notes, include a comprehensive list of parking lot actions. Parking lot actions, or parking lots, or open topics . Parling lot or open topics are topics that cannot be covered in the current meeting and need to be captured for later action. Parking lots include any activities that are either explicitly stated or implicitly stated or suggested, as an action to be taken, follow-up activity, or next step. Look for the following common phrases or terms that can be used to identify the list of action items: ""parking lot"", ""action item"", ""follow-up"", ""let me take that"", ""let me get back with you"", ""check on that"", ""Open Topic"", ""Open Point"", ""need to check"", ""not sure"", ""send that"", ""let me confirm"", ""table that"", etc. For each parking lot item, include the target completion date and who is responsible . if this information is included or can be determined in the meeting transcript. In another outline Issues and the corresponding action , or a something that was highlighted as challenge to which a recommendation or solution was provided or discussed, capture this also under Issues and Actions section. Capture the customer process pain points in another section. Pain points are challenges within the current processes that are typically manually intensive or inadequate, something not working right , something stated as a challenge etc.  In another section, include the high-level requirements that include any stated capability, functionality or process that is required for the new solution. In another section create a Summary as bullet points on what was discussed in the meeting.  Review the transcript from the beginning to the end and then re-read the transcript from the end back to the start to ensure that no details are missed. The meeting notes should only include information determined from the attached document, so do not add or make up any information that may be missing. Use the instructions for the content in the attached document."

In [ ]:
def chatty(message, history):
    files = message["files"]

    #if files is empty or the directory db_name does not exist, return an error message
    # if not os.path.exists(db_name) and not files: 
    #     return "No files uploaded. Please upload a file(s) containing the meeting transcripts."
  
    if files:
        vectorstore = createChunks(files)
        retriever = vectorstore.as_retriever()
   
        # the retriever is an abstraction over the VectorStore that will be used during RAG
        # retriever = vectorstore.as_retriever()

        # Initialize retriever - either with namespace filter or without
    if retriever:
        
        # set up the conversation memory for the chat
        memory = ConversationBufferMemory(memory_key='chat_history', output_key='answer', return_messages=True)
        
        # putting it together: set up the conversation chain with the GPT 4o LLM, the vector store and memory
        conversation_chain = ConversationalRetrievalChain.from_llm(llm=chat_llm, retriever=retriever, memory=memory)
        #prompty = message["text"] + "/n" + prompt1     
        result = conversation_chain.invoke({"question":message["text"]})
        return result["answer"]
    else:
        return "No files uploaded. Please upload a file(s) containing the meeting transcripts."

In [ ]:
#For uploads
######################################################
from pathlib import Path
from gradio_client import Client, FileData, handle_file
import shutil

with gr.Blocks() as demo:
    gr.ChatInterface(
        chatty,
        type="messages",
        title="RAG Chatbot",
        description="Upload text(.txt) files and ask questions about them!",
        textbox=gr.MultimodalTextbox(file_types=[".txt"], file_count="multiple"),
        multimodal=True
    )

demo.launch()

Using Langchain

In [ ]:
%pip install langchain
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:

from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [ ]:
def process_files(files):
    # contents = []
    # for file in files:
    #     with open(file.name, 'r') as f:
    #         contents.append(f.read())
    # return "\n".join(contents)  
    # ##

    global conv_chain
    print(files)
    vs = createChunks(files)
    print(vs._collection.count())
    ###################################################################
    retriever1 = vs.as_retriever()
      
    # set up the conversation memory for the chat
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    
    # putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
    conv_chain = ConversationalRetrievalChain.from_llm(llm=chat_llm, retriever=retriever1, memory=memory)
            
    
###############################################################################################
    return vs

Trial

In [ ]:
import random
import time

#For uploads
from pathlib import Path

def upload_file(filepath):
    name = Path(filepath).name
    print(f"File uploaded: {name}")
    
    
# def process_files(files):
#     # contents = []
#     # for file in files:
#     #     with open(file.name, 'r') as f:
#     #         contents.append(f.read())
#     # return "\n".join(contents)    
#     vs = createChunks(files)

context = "Key points:\n- EY has two cloud platform enterprise agreements (CPEAs). One is for the Mercury landscape and the other is for their Symphony landscape.\n- The CPEA for Mercury is projected to exhaust the allocated credits by end of March.  However, the CPEA for Symphony has approximately $8 million worth of credits that need to be used by June 2026.\n- Options are being considered to right size the two CPEAs or possibly merge them to make the best use of the total overall credits owned by EY and avoid losing unused credits.\n- Based on the discussion, merging of two CPEAs is a significant undertaking.  The process is outlined in SAP Note 3246456, but we need to better understand the prerequisites and impacts.\n- Sandeep has opened a ticket with the Cloud Operations team to initiate discussion on the topic.\n\nAdditional information:\n- The BTP accounts for Symphony primarily use Neo.  SAP is ending support for Neo on 12/31/2028 according to SAP Note 3351844.  Customers need to move to Cloud Foundry before 12/31/2028.  So far, EY has not been interested in engaging in this topic.  \n- The BTP accounts for Mercury are already using Cloud Foundry.\n- The Mercury CPEA ends in August.\n\nNext steps:\n\tSandeep will follow up on the Cloud Operations ticket that was opened and attempt to identify someone on that team to discuss the process further.\nOnce we have enough information, this team can regroup and present the consolidation option to EY, if feasible.\n"


# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(type="messages")
#     msg = gr.Textbox()
#     clear = gr.ClearButton([msg, chatbot])

#     def respond(message, chat_history):
#         bot = ChatBot(orchestration_service=orchestration_service, context=context)
#         bot_message = bot.chat(message)
#         chat_history.append({"role": "user", "content": message})
#         chat_history.append({"role": "assistant", "content": bot_message})
#         #time.sleep(2)
#         return "", chat_history

#     msg.submit(respond, [msg, chatbot], [msg, chatbot])

#     # Upload
#     # with gr.Row():
#     #     u = gr.UploadButton("Upload files", file_count="multiple")
#     # u.upload(upload_file, u)

#     u =gr.Files(label="Upload multiple files")
#     u.upload(process_files, u)

# demo.launch()
##################################################################################################################


In [ ]:
def respond(message, chat_history):
    bot = ChatBot(orchestration_service=orchestration_service, context=context)
    bot_message = bot.chat(message)
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": bot_message})
    return "", chat_history
# UI Setup with Gradio
with gr.Blocks() as demo:
    gr.Markdown("# RAG Chatbot with File Upload")
    gr.Markdown("Upload a text file to chat with it or simply chat without uploading a file.")
    
    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.Files(label="Upload text file (optional)")
            upload_button = gr.Button("Process File")
            file_status = gr.Textbox(label="Upload Status", interactive=False)
        
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(type="messages", height=500)
            msg = gr.Textbox(label="Message", placeholder="Type your question here...")
            send = gr.Button("Send")
    
    # Handle file upload
    upload_button.click(
        fn=process_files,
        inputs=[file_input],
        # outputs=[file_input, file_status]
    )
    
    # Handle chat message
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

# Launch the Gradio app
demo.launch()

Do it properly, no more mucking around